In [1]:
from collections import OrderedDict
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import struct

In [2]:
import ast

def bit_conversion(string):
  bytes_value = ast.literal_eval(string)
  bin = np.frombuffer(bytes_value, dtype=np.uint8)
  bin = np.unpackbits(bin, axis=0).astype(np.single)
  return bin

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

df = pd.read_csv('evals.csv')
df.drop(df.tail(10).index,inplace=True)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
# set data and device

''' device = 'cuda' if torch.cuda.is_available() else 'cpu'

df = pd.read_csv('evals.csv')
df.drop(df.tail(10).index,inplace=True) '''

df = df.head(1000000)

df['bin'] = df['binary'].apply(bit_conversion)
df['bound_evals'] = df['eval'].clip(upper=15, lower=-15)

X = df['bin']
y = df['bound_evals']

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(dim=1)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, y_tensor,
    test_size=0.2,
    random_state=42
)

In [ ]:
# Create datasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=1024,
    shuffle=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=1024,
    shuffle=False
)

In [ ]:
class ChessCNN(nn.Module):
    def __init__(self, learning_rate=1e-3, batch_size=1024, layer_count=10):
        super().__init__()

        layers = []
        for i in range(layer_count - 1):
            layers.append((f"linear-{i}", nn.Linear(808, 808)))
            layers.append((f"relu-{i}", nn.ReLU()))
        layers.append((f"linear-{layer_count-1}", nn.Linear(808, 1)))
        self.seq = nn.Sequential(OrderedDict(layers))

    def forward(self, x):
        return self.seq(x)

In [ ]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):

  chess_model.train()

  train_loss = 0

  for batch, (batch_X, batch_y) in enumerate(train_loader):
      batch_X, batch_y = batch_X.to(device), batch_y.to(device)

      y_pred = chess_model(batch_X)
      loss = loss_fn(y_pred, batch_y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      train_loss = loss

  return train_loss

In [ ]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module):

  model.eval()

  test_loss = 0

  with torch.inference_mode():
    for batch, (batch_X, batch_y) in enumerate(dataloader):
      batch_X, batch_y = batch_X.to(device), batch_y.to(device)

      y_preds = chess_model(batch_X)

      y_loss = loss_fn(y_preds, batch_y)

      test_loss = y_loss

  return test_loss

In [ ]:
from tqdm.auto import tqdm

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.L1Loss(),
          epochs: int = 10):

  results = {
      "train_loss": [],
      "test_loss": [],
  }

  for epoch in tqdm(range(epochs)):
    train_loss = train_step(model=chess_model,
                            dataloader=train_dataloader,
                            loss_fn=loss_fn,
                            optimizer=optimizer)

    test_loss = test_step(model=model,
                          dataloader=train_dataloader,
                          loss_fn=loss_fn)

    print(f"Epoch: {epoch+1} |"
      f"train_loss: {train_loss:.4f} |"
      f"test_loss: {test_loss:.4f} "
    )

    results["train_loss"].append(train_loss.item() if isinstance(train_loss, torch.Tensor) else train_loss)
    results["test_loss"].append(test_loss.item() if isinstance(test_loss, torch.Tensor) else test_loss)

  return results

In [ ]:
chess_model = ChessCNN().to(device)
chess_model

In [ ]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(params=chess_model.parameters(), lr=0.001)

In [ ]:
from timeit import default_timer as timer
start_time = timer()

NUM_EPOCHS = 400

chess_model_results = train(model=chess_model,
                        train_dataloader=train_loader,
                        test_dataloader=test_loader,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS)

end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "chess_model.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=chess_model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH)

from google.colab import files
files.download(MODEL_SAVE_PATH)

In [ ]:
''' from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "chess_model.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=chess_model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH)

from google.colab import files
files.download("data.csv") '''